In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [ ]:
# import IPython
# import sys
# from pygaggle.rerank.transformer import MonoT5
# !{sys.executable} -m pip install -r pygaggle/requirements.txt
# IPython.Application.instance().kernel.do_shutdown(True)  

In [ ]:
# """ Generate Taskgraphs from datasets """

# from models_datasets.recipe_1m_model import AbstractModelDataset
# from models_datasets.recipe_1m_model import Recipe1MModel
# from models_datasets.wikihow_model import WikihowModel

# dataset_models = {
#     "COOKING" : Recipe1MModel,
#     "DIY": WikihowModel,
# }

# for model_name, model in dataset_models.items():
#     dataset_model: AbstractModelDataset = model()
#     print(f"Dataset Model: {model_name}")
#     dataset_model.convert_to_taskgraphs()

In [2]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()

,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [2]:
from models_indexes.bm25_grid_search import CustomRetrievalModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, average_precision_score
import numpy as np
import pandas as pd

# model = Bm25ModelGridCV(domain = "COOKING", queries = queries["COOKING"])
# model.set_k1_range(0.2,1, 0.2)
# model.set_b_range(0.1, 0.3, 0.1)
# model.fit()

# Load annotated data
annotated_data = pd.read_csv('/home/ubuntu/task-search-quality/queries/cooking.csv')

print(annotated_data)

# # Split annotated data into training and test sets
# train_data = annotated_data.sample(frac=0.8, random_state=42)
# test_data = annotated_data.drop(train_data.index)

# Define the parameter grid to search over
param_grid = {
    'k1': np.arange(1.2, 2.0, 0.1),
    'b': np.arange(0.5, 1.5, 0.1),
}

# Define the GridSearchCV object
grid_search = GridSearchCV(CustomRetrievalModel(), param_grid, cv=4, scoring=make_scorer(average_precision_score))

# # Fit the GridSearchCV object to the training data
grid_search.fit(train_data['query'], train_data['doc_id'])

# # Print the best hyperparameters and score on the test set
# print("Best hyperparameters:", grid_search.best_params_)
# print("Mean average precision (MAP) score on test set:", grid_search.score(test_data['query'], test_data['doc_id']))


          id                                        raw query   domain  \
0    query-0                       how to spatchcock a turkey  cooking   
1    query-1     I want an easy to make dessert for christmas  cooking   
2    query-2  any recommendations for a gluten free appetizer  cooking   
3    query-3                how to make a green goddess salad  cooking   
4    query-4             how to make a classic english trifle  cooking   
..       ...                                              ...      ...   
95  query-95                    spicy vegetarian indian curry  cooking   
96  query-96                       a grilled filling sandwich  cooking   
97  query-97                healthy sweet potato french fries  cooking   
98  query-98                             a well balanced meal  cooking   
99  query-99            make chicken soup with homemade broth  cooking   

   knowledge complex many points of view different key entities seasonal  \
0          n       y               

In [ ]:
# from models_indexes.bm25_model import BM25Model
# from models_indexes.ance_model import AnceModel
# from models_indexes.colbert_model import ColbertModel
from models_indexes.marqo_model import MarqoModel
# from models_indexes.abstract_model import AbstractModel
# from models_indexes.hybrid_model import HybridModel

models = {}

# for domain in  ["DIY"]:
for domain in  ["DIY", "COOKING"]:
    print(domain)
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance+t5": AnceModel(domain = domain, t5=True),
        # "colbert+t5": ColbertModel(domain = domain, t5=True),
        "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain, doc_build="separate"),
        # "hybrid": HybridModel(domain = domain, rm3=True, t5=True)
    }

In [22]:
            # "_id": taskmap.taskmap_id,
            # "Title": taskmap.title,
            # "Date": taskmap.date,
            # "Steps": self.__parse_steps(taskmap),
            # "Tags": self.__parse_tags(taskmap),
            # "Requirements": self.__parse_requirements(taskmap),
            # "Domain": taskmap.domain_name,
            # "Description": taskmap.description,
from models_indexes.marqo_model import MarqoModel
model = MarqoModel(domain = "COOKING", t5 = False)
qs = queries["COOKING"]

# # model.convert_search_results_to_run(qs)
# resp1 = model.search("pasta")
# resp2 = model.search("fix kitchen",["Title"])
# resp2

# model.convert_search_results_to_run_attributes(qs, filters=["Tags"])
# model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps"])
# model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps", "Title"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Requirements"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps" ,"Requirements"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Steps" ,"Requirements"])
# model.convert_search_results_to_run_attributes(qs, filters=["Steps"])
# model.convert_search_results_to_run_attributes(qs, filters=["Steps", "Title"])
# model.convert_search_results_to_run_attributes(qs, filters=["Title"])



query 0/100
FILTERS ADDED ['Title', 'Tags']


2023-02-21 19:39:13,035 logger:'marqo' INFO search (tensor): took 0.979s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.972s to execute the search.
2023-02-21 19:39:13 [INFO] index: search (tensor): took 0.979s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.972s to execute the search.


query 1/100
FILTERS ADDED ['Title', 'Tags']


2023-02-21 19:39:13,256 logger:'marqo' INFO search (tensor): took 0.220s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.215s to execute the search.
2023-02-21 19:39:13 [INFO] index: search (tensor): took 0.220s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.215s to execute the search.
2023-02-21 19:39:13,415 logger:'marqo' INFO search (tensor): took 0.158s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.152s to execute the search.
2023-02-21 19:39:13 [INFO] index: search (tensor): took 0.158s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.152s to execute the search.


query 2/100
FILTERS ADDED ['Title', 'Tags']
query 3/100
FILTERS ADDED ['Title', 'Tags']


...

query 95/100
FILTERS ADDED ['Title', 'Tags']
query 96/100
FILTERS ADDED ['Title', 'Tags']


2023-02-21 19:39:26,252 logger:'marqo' INFO search (tensor): took 0.132s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.126s to execute the search.
2023-02-21 19:39:26 [INFO] index: search (tensor): took 0.132s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.126s to execute the search.
2023-02-21 19:39:26,362 logger:'marqo' INFO search (tensor): took 0.108s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.103s to execute the search.
2023-02-21 19:39:26 [INFO] index: search (tensor): took 0.108s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.103s to execute the search.


query 97/100
FILTERS ADDED ['Title', 'Tags']
query 98/100
FILTERS ADDED ['Title', 'Tags']


2023-02-21 19:39:26,487 logger:'marqo' INFO search (tensor): took 0.124s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.119s to execute the search.
2023-02-21 19:39:26 [INFO] index: search (tensor): took 0.124s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.119s to execute the search.
2023-02-21 19:39:26,606 logger:'marqo' INFO search (tensor): took 0.118s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.113s to execute the search.
2023-02-21 19:39:26 [INFO] index: search (tensor): took 0.118s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.113s to execute the search.


query 99/100
FILTERS ADDED ['Title', 'Tags']
Run file saved at /home/ubuntu/task-search-quality/measurements/diy/run_files/marqo-filters/Title-Tags.run


In [ ]:
from models_indexes.bm25_model import BM25Model

model = BM25Model(domain = "DIY")

model.search_best("clean bathroom")

In [ ]:
resp2
# build indexes
# print("Build indexes")
# for domain, index_models in models.items():
#     for model_name, model in index_models.items():
#         model.build_index()

In [ ]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)